In [1]:
import pandas as pd
import csv 
import keras
from keras.layers import Input
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from tensorflow.keras.utils import to_categorical
from keras.callbacks import TensorBoard, Callback
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

In [2]:
np.set_printoptions(precision=3, suppress=True)


In [3]:
train_df = pd.read_csv(r"TaiLieu/train-0.csv", header=None, skiprows=1).sample(frac=1)
val_df   = pd.read_csv(r"TaiLieu/file-0.csv", header=None, skiprows=1).sample(frac=1)
test_df  = pd.read_csv(r"TaiLieu/file-1.csv", header=None, skiprows=1).sample(frac=1)

In [4]:
train_y = np.array(train_df.iloc[:, 2]) #lấy cột 2 là cột nhãn 
train_x = np.array(train_df.iloc[:,3:]) #lấy cột 3 làm đặc trưng
unique, counts = np.unique(train_y, return_counts=True)
print("LABEL TRAIN: ", len(unique))

val_y = np.array(val_df.iloc[:, 2]) #lấy cột 2 là cột nhãn 
val_x = np.array(val_df.iloc[:,3:]) #lấy cột 3 làm đặc trưng
unique_v, counts_ = np.unique(val_y, return_counts=True)
print("LABEL VAL: ", len(unique_v))

test_y = np.array(test_df.iloc[:, 2]) #lấy cột 2 là cột nhãn 
test_x = np.array(test_df.iloc[:,3:]) #lấy cột 3 làm đặc trưng
unique_t, counts_t = np.unique(test_y, return_counts=True)
print("LABEL TEST: ", len(unique_t))

LABEL TRAIN:  156
LABEL VAL:  222
LABEL TEST:  179


In [5]:

BATCH_SIZE = 32
SIZE = 1908
SIZE2= 1
N_CLASSES = 228
LR = 0.001
N_EPOCHS = 15

print(train_x.shape,train_y.shape)
print(val_x.shape,val_y.shape)
print(test_x.shape,test_y.shape)


(25713, 1908) (25713,)
(3260, 1908) (3260,)
(3217, 1908) (3217,)


In [6]:

train_x = train_x.reshape(train_x.shape[0], SIZE2, SIZE, 1)
val_x = val_x.reshape(val_x.shape[0], SIZE2, SIZE, 1)
test_x = test_x.reshape(test_x.shape[0], SIZE2, SIZE, 1)

LABELS = np.unique(train_y)
original_test_y = test_y

train_y = to_categorical(train_y, N_CLASSES)
val_y = to_categorical(val_y, N_CLASSES)
test_y = to_categorical(test_y, N_CLASSES)

print(LABELS)


[  0   1   4   5   6  12  13  15  18  22  24  25  26  31  32  33  34  35
  36  39  40  41  42  43  44  46  47  48  49  51  52  54  56  58  60  62
  63  64  66  67  68  69  71  72  73  74  75  76  77  78  79  83  84  85
  87  88  89  92  95  98 100 102 104 106 108 109 110 111 112 116 117 120
 121 123 124 125 126 130 132 133 134 139 142 145 148 149 150 151 152 153
 155 156 157 158 160 161 163 164 166 167 169 171 172 173 175 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227]


In [7]:

input0 = Input(shape=(SIZE2,SIZE,1))
conv1 = Conv2D(32, kernel_size=2, activation='relu', padding="same" ,input_shape=(SIZE2, SIZE, 1))(input0)
pool1 = MaxPooling2D((2, 2), padding = 'same')(conv1)
conv2 = Conv2D(32, kernel_size=2, activation='relu', padding="same")(pool1)
pool2 = MaxPooling2D((2, 2), padding = 'same')(conv2)

conv3 = Conv2D(64, kernel_size=2, activation='relu', padding="same")(pool2)
pool3 = MaxPooling2D((2, 2), padding = 'same')(conv3)
'''
conv4 = Conv2D(64, kernel_size=2, activation='relu', padding="same")(pool3)
pool4 = MaxPooling2D((2, 2), padding = 'same')(conv4)
'''
flatten_per = Flatten()(pool3)

hidden1 = Dense(1024, activation='relu')(flatten_per)
output = Dense(N_CLASSES, activation='softmax')(hidden1)

model = Model(inputs=input0, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
class Metrics(Callback):
    def __init__(self, x, y):
        self.x = x
        self.y = y if (y.ndim == 1 or y.shape[1] == 1) else np.argmax(y, axis=1)
        self.reports = []

    def on_epoch_end(self, epoch, logs={}):
        y_hat = np.asarray(self.model.predict(self.x))
        y_hat = np.where(y_hat > 0.5, 1, 0) if (y_hat.ndim == 1 or y_hat.shape[1] == 1)  else np.argmax(y_hat, axis=1)
        report = classification_report(self.y,y_hat,output_dict=True)
        self.reports.append(report)
        return
   
    # Utility method
    def get(self, metrics, of_class):
        return [report[str(of_class)][metrics] for report in self.reports]

In [ ]:
metrics_multiclass = Metrics(train_x, train_y)
history = model.fit(train_x, train_y, epochs=N_EPOCHS, batch_size = BATCH_SIZE ,validation_data=(val_x, val_y), callbacks=[metrics_multiclass])


Epoch 1/15
804/804 [==============================] - 187s 232ms/step - loss: 0.5169 - accuracy: 0.8765 - val_loss: 0.4856 - val_accuracy: 0.9166
Epoch 2/15


C:\Users\Echidna\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Echidna\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Echidna\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

424/804 [==============>...............] - ETA: 1:16 - loss: 0.1296 - accuracy: 0.9529